# **Introduction: Location! Location! Location!**
One of the most critical factors to a restaurants success is its location. However finding a good location for a restaurant is not always as easy as it seems. 


I will be using data from the foursquare api to provide a recomendation as to locations that could be advantageous to restaurants looking to open in Las Vegas. I plan to use foursquare data to identify popular venus in the city that are not near other popular venues. The idea is that new restaurants could open next to these island popular venues, and attract more foot traffic and visibility. 

**Audience:** This problem will be of particular interest to investors, chain restaurts, or restuaranters interested in opening a new restaurants in las vegas.

# **Data**
The following data points will be used to solve the problem of where to open a restaurant in Las vegas are as follows:


*   A list of restaurants within a 5 mile radius
*   A list of all venues within a 5 mile radius
*   The number of likes a venue has received in the last year





 First we will add all the dependencies needed to get the data

In [ ]:
import pandas as pd
import json
from geopy.geocoders import Nominatim
import requests 
from pandas.io.json import json_normalize

First we need to use Geopy to get the coordinates for Las Vegas

In [ ]:
#First we need to get the coordinates for Toronto from Geopy
address = 'Las Vegas'
geolocator = Nominatim(user_agent="t_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Las Vegas are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Las Vegas are 36.1672559, -115.148516.


Now lets connect to the Foresauare API to find the most popular venues in the city

In [ ]:
#@title Foresquare Credentials
CLIENT_ID = 'IVHAAZGAO005CDQNYE5QLH4R3NPCCCFYFBY5Z44OSQOVQNSF'
CLIENT_SECRET = 'XSLPV044NAKZEUCS0POWXAMV2UEUPX2KOF3T4II0OZ0TYPWB'
VERSION = '20180605' 
LIMIT = 100   

In [ ]:
radius = 8046 #this number is in meters, 5 miles = 8046 meters.
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()


Lets put the results into a dataframe to make it easier to work with


---



In [ ]:
venues = results['response']['groups'][0]['items']
full_df = pd.json_normalize(venues)
#filter the columns
columns = ['venue.id','venue.name','venue.location.formattedAddress','venue.location.lat','venue.location.lng','venue.location.distance','venue.categories']
df = full_df.loc[:,columns]


Let's clean up that category column. We are only interested in the PluralName



In [ ]:
#Loop through the column and get the pluralName for each row
names = []
for x in df['venue.categories']:
  n = x[0]['shortName']
  names.append(n)
#add the pluralName as a new column
df['category'] = names


We also need to get the amount of likes each location has received, and add it to the dataframe. First lets get the Venue Id information from the dataframe, and put it in a list


In [ ]:
# get the Venue Id information from the dataframe, and put it in a list
id_list = df['venue.id'].tolist()

Next we will loop through the list and connect to the /likes endpoint to get the likes count for each venue


In [ ]:
#@title Token
token = 'T1U2OD3W1OFGBXMF1I44Z1RGCYVUY1HQIX324AY3DCMQJ11Y'

In [ ]:
#this loop will get the likes for each venue ID and add them to a list, which will then be added to the dataframe
likes_list = []
for x in id_list:
  stats_url = 'https://api.foursquare.com/v2/venues/'+ x +'/likes?&oauth_token={}&client_id={}&client_secret={}&v={}'.format(
      token,
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION)
  stats_json = requests.get(stats_url).json()
  likes =  stats_json['response']['likes']['count']
  likes_list.append(likes)

# **Methodology**

Now that we have the data we need, lets explore it. We are going to be looking for venues that have a lot of likes, but are not near other popular venues. We will accomplish this by identifying what is considred "popular" by looking at the quartiles of the likes, and then creating a map with color coded markeres, indicating where each restaurant is and if it is popular or not. We will use the quartiles of the likes data for bins in our color code.

In [ ]:
import numpy as np
#add the likes list to the dataframe
df['venue likes'] = likes_list

#lets explore the like data a little bit
lq1 = np.quantile(likes_list, .25)
lq2 = np.quantile(likes_list, .50)
lq3 = np.quantile(likes_list, .75)
lq4 = np.quantile(likes_list, 1)

print("Q1 quantile of likes list is ", lq1)
print("Q2 quantile of likes list is ", lq2)
print("Q3 quantile of likes list is ", lq3)
print("Q4 quantile of likes list is ", lq4)

Q1 quantile of likes list is  61.0
Q2 quantile of likes list is  117.5
Q3 quantile of likes list is  249.75
Q4 quantile of likes list is  7578


Ok we have the information we need. Lets see how many instances of each category there are


In [ ]:
df['category'].value_counts()

Next map where the restaurants are, with color codes indicating which ones have the most likes. The colors in ascending number of likes will go blue, purple, green, red. Meaning blue will have the fewest likes and red the most.

In [ ]:
import folium

lat_list = df['venue.location.lat']
long_list = df['venue.location.lng']
names_list = df['venue.name']
map = folium.Map(location = [latitude, longitude])
purple = "purple"
blue = "blue"
green = "green"
red = "red"

for x, y, n, l in zip(lat_list, long_list, names_list, likes_list):
  name = n
  if l <  60:
    folium.Marker([x, y], tooltip=n, icon = folium.Icon(color = "blue")).add_to(map)
  elif 61 <= l <= 118:
    folium.Marker([x, y], tooltip=n, icon = folium.Icon(color = "purple")).add_to(map)
  elif 119 <= l <= 241:
    folium.Marker([x, y], tooltip=n, icon = folium.Icon(color = "green")).add_to(map)
  else:
    folium.Marker([x, y], tooltip=n, icon = folium.Icon(color = "red")).add_to(map)
map

Now that we have a visualization of where the most popular restaurants and venues are, we want to look for venues that are popular, yet not near other venues. This is where we will want to open a new restaurant. 

# **Results & Discussion**

We can see from the map there are a few venues that seem to be popular, yet not surrounded by too many other venues. For example: Tacos el Gordo, Firefly Tapas Kitchen & Bar, Cashman Field, Raising Canes, and Pop's Philly Steaks all might make for good venus to open next too. They have a relatively high amount of likes without too much competition nearby.

The highest concentration of popular venus seems to be near the Wynn Golf Course, while a cluster of unpopular restaurants lies north of the at area.

# **Conclusion**

The purpose of this report was to identify popular locations for a resutarunt. One might also want to consider the types of venues that are near by, as some may be more conduscive to consumers looking for a place to eat. 

While the competition for restaurants in Las Vegas is strong, there appears to be room for more still. Highly competitive markets mean there is strong demnad for good products. Opening next to popular venus will not only help in driving awarness and foot traffic, but will also help to associate your brand with other popular brands.

Thank you for reading this notebook